In [6]:
import pandas as pd
import requests
from datetime import datetime
import folium
from IPython.display import IFrame
from flask import Flask, render_template

In [7]:
country_geo = 'world-countries.json'
re = requests.get("https://corona.lmao.ninja/v2/countries?yesterday&sort")
re = re.json()
re = pd.DataFrame.from_dict(re)
country_info = re['countryInfo']
country_info = pd.json_normalize(country_info)
df = pd.concat([re, country_info], axis=1,sort=False)
df = df.drop('countryInfo', axis=1)
# print(df)

In [8]:
def top_ten_countries(n=10):
    by_country = df.groupby('country').sum()[['cases']]
    top_ten = by_country.nlargest(n, 'cases')[['cases']]
    top_ten['cases'] = top_ten['cases'].apply(lambda x: "{:,}".format(x))
    return top_ten

d = top_ten_countries()

In [9]:
def convert_time_stamp(x):
    t = datetime.fromtimestamp(x/1000.0)
    s = t.strftime('%Y-%m-%d %H:%M:%S')
    return s[:-3]
df['updated'] = df['updated'].apply(convert_time_stamp)

In [10]:
df['recovered'] = df['recovered'].apply(lambda x: "{:,}".format(x))
df['cases'] = df['cases'].apply(lambda x: "{:,}".format(x))
df['deaths'] = df['deaths'].apply(lambda x: "{:,}".format(x))

def enter_country(t):
    country = t.title()
    country.strip()
    x = df[df['country'] == country]
    if x.empty:
        return "Sorry, No current news on this country"
    else:
        return x[['country', 'cases', 'todayCases']]
test = enter_country('zambia')
print(test)

TypeError: unsupported format string passed to Series.__format__

In [ ]:
m = folium.Map(
    location=[100, 0],
    zoom_start=1.5,
)

In [ ]:
folium.Choropleth (
    geo_data= country_geo,
    data=df,
    columns=['lat', 'long'],
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Covid Analysis'
).add_to(m)

In [ ]:
def circle_maker(x):
    folium.Circle(location=[x[0],x[1]],
                 radius=10,
                 color="red",
                 popup='{}\n Cases: {}\n Deaths: {}\n Recovered: {}'.format(x[2], x[3], x[4], x[5])).add_to(m)
df[['lat','long','country','cases','deaths', 'recovered']].apply(lambda x:circle_maker(x),axis=1)
html_map = m._repr_html_()

In [ ]:
IFrame(src=m, width=450, height=450)
folium.Popup(src=m, max_width=1000)

In [ ]:
app = Flask(__name__)

@app.route('/')
def home():
    return render_template("home.html", cmap=html_map)

@app.route('/about')
def about():
    return render_template("about.html")

if __name__ == "__main__":
    app.run(debug=True)
